<a href="https://colab.research.google.com/github/Pradeep23-01/General-CNN-models/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
#Keras library for CIFAR dataset
from keras.datasets import cifar10
(x_train, y_train),(x_test, y_test)=cifar10.load_data()


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.3)

In [ ]:
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))


In [ ]:
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

In [ ]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [ ]:
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation


#Initializing CNN
cnn=tf.keras.models.Sequential()

#1st Convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=96 ,kernel_size=(11,11), strides=(4,4), input_shape=(32,32,3)))

cnn.add(BatchNormalization())
cnn.add(Activation('relu'))

#Overlapping Max POOL
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

#2nd COnvolution Layer
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), padding='same'))

cnn.add(BatchNormalization())
cnn.add(Activation('relu'))

#Max POOl
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

#3rd Convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=384, kernel_size=3, strides=1, padding='same'))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))

#4th convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=384, kernel_size=3, strides=1, padding='same'))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))

#5th Convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same'))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))

#Max POOL
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Dropout
cnn.add(tf.keras.layers.Dropout(0.5))

#Flattening
cnn.add(tf.keras.layers.Flatten())

#Full COnnection
cnn.add(tf.keras.layers.Dense(units=4096,input_shape=(32,32,3)))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))

#Dropout
cnn.add(tf.keras.layers.Dropout(0.5))

#Full COnnection
cnn.add(tf.keras.layers.Dense(units=4096))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))
cnn.add(tf.keras.layers.Dropout(0.5))



#Full COnnection
cnn.add(tf.keras.layers.Dense(units=1000))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))
cnn.add(tf.keras.layers.Dropout(0.5))



cnn.add(tf.keras.layers.Dense(10))
cnn.add(BatchNormalization())
cnn.add(Activation('softmax'))


cnn.summary()



Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 6, 6, 96)          34944     
_________________________________________________________________
batch_normalization_59 (Batc (None, 6, 6, 96)          384       
_________________________________________________________________
activation_60 (Activation)   (None, 6, 6, 96)          0         
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 3, 3, 96)          0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 3, 3, 256)         614656    
_________________________________________________________________
batch_normalization_60 (Batc (None, 3, 3, 256)         1024      
_________________________________________________________________
activation_61 (Activation)   (None, 3, 3, 256)       

In [ ]:
cnn.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(   monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5) 

In [ ]:
batch_size= 100
epochs = 6
learn_rate=.001

cnn.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = 250, callbacks = [lrr], verbose=1)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/6
350/350 [==============================] - 487s 1s/step - loss: 1.8790 - accuracy: 0.3237 - val_loss: 2.2519 - val_accuracy: 0.3093
Epoch 2/6
350/350 [==============================] - 458s 1s/step - loss: 1.4607 - accuracy: 0.4930
Epoch 3/6
350/350 [==============================] - 472s 1s/step - loss: 1.3400 - accuracy: 0.5406
Epoch 4/6
350/350 [==============================] - 466s 1s/step - loss: 1.2525 - accuracy: 0.5733
Epoch 5/6
350/350 [==============================] - 466s 1s/step - loss: 1.1883 - accuracy: 0.5947
Epoch 6/6
350/350 [==============================] - 465s 1s/step - loss: 1.1178 - accuracy: 0.6210
